# Project 5: Optimizing Evacuation Routes using Real-Time Traffic Information

Kelly Slatery | US-DSI-10 | 02.21.2020

## Model Data

In [1]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, \
                             GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix

# from sklearn.naive_bayes import MultinomialNB
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC

# Revise this imports list
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

## Import Data

In [2]:
df = pd.read_csv('./data/train_data/final_train_data.csv')
df.shape

(165204, 4)

In [3]:
df.head()

,dates&time,user,tweet,class
0,2020-02-06 22:03:12+00:00,DallasPD,DallasPD and dfrincidents are currently on loc...,1
1,2019-08-30 21:16:20+00:00,DallasPD,Monday Sept on LaborDay Jack Evans Police Hd...,1
2,2019-08-14 22:31:39+00:00,DallasPD,PIODPD is at the scene of a possible barricade...,1
3,2019-07-13 22:04:27+00:00,DallasPD,PIODPD is on scene of an Officer Involved Shoo...,1
4,2019-07-13 01:38:19+00:00,DallasPD,Major police incident in downtown Dallas Griff...,1


In [4]:
# Check for nulls
df.isnull().sum()

dates&time    0
user          0
tweet         0
class         0
dtype: int64

In [5]:
# Check out the y value distribution
df['class'].value_counts(normalize=True)

0    0.948833
1    0.051167
Name: class, dtype: float64

Baseline Accuracy: 95%

In [6]:
# Define X and y
X = df['tweet']
y = df['class']

# Train tes split data
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.25, 
                                                    stratify=y,
                                                    random_state=42)

## Logistic Regression: CountVecotorizer

In [7]:
# Set up pipeline
lr_cv_pipe = Pipeline([
    ('cvec', CountVectorizer()), 
    ('lr', LogisticRegression(solver='liblinear')) 
])

# Set up pipeline parameters
lr_cv_params = {
    'cvec__stop_words': [None, 'english'],
    'cvec__ngram_range': [(1,1), (1,2), (1,3)], 
    'cvec__max_features': [100, 500], 
    'cvec__max_df': [1.0, .95], 
    'cvec__min_df': [1, .05], 
    'lr__C': [1, 1e9], 
    'lr__penalty': ['l1', 'l2'],
    'lr__max_iter': [100, 500, 1000]
}

In [8]:
# Set up gridsearch
lr_cv_gs = GridSearchCV(lr_cv_pipe, lr_cv_params, n_jobs=4, cv=5, verbose=1)

# Fit the gridsearch to the training data
lr_cv_gs.fit(X_train, y_train);

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   38.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  3.4min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 10.2min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 18.7min
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed: 30.2min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed: 45.8min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed: 63.8min
[Parallel(n_jobs=4)]: Done 2880 out of 2880 | elapsed: 75.0min finished


In [9]:
# What was our highest cross-val accuracy score across model hyperparameter combinations?
lr_cv_gs.best_score_

0.9971509971509972

In [10]:
# What model hyperparameters yielded the highest accuracy score?
lr_cv_gs.best_estimator_

Pipeline(memory=None,
         steps=[('cvec',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=500, min_df=1, ngram_range=(1, 2),
                                 preprocessor=None, stop_words=None,
                                 strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('lr',
                 LogisticRegression(C=1, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,
                              

In [11]:
# What were the parameters?
lr_cv_gs.best_params_

{'cvec__max_df': 1.0,
 'cvec__max_features': 500,
 'cvec__min_df': 1,
 'cvec__ngram_range': (1, 2),
 'cvec__stop_words': None,
 'lr__C': 1,
 'lr__max_iter': 100,
 'lr__penalty': 'l1'}

In [12]:
# How does this model perform on the test set?
lr_cv_gs.score(X_test, y_test)

0.9972639887654052

This model performs with high accuracy on both the train and test datasets. It is neither overfit nor underfit.

## Logistic Regression: TfidfVecotorizer

In [13]:
# Set up pipeline
lr_tv_pipe = Pipeline([
    ('tvec', TfidfVectorizer()), 
    ('lr', LogisticRegression(solver='liblinear')) 
])

# Set up pipeline parameters
lr_tv_params = {
    'tvec__stop_words': [None, 'english'],
    'tvec__ngram_range': [(1,1), (1,2), (1,3)], 
    'tvec__max_features': [100, 500], 
    'tvec__max_df': [1.0, .95], 
    'tvec__min_df': [1, .05], 
    'lr__C': [1, 1e9], 
    'lr__penalty': ['l1', 'l2'],
    'lr__max_iter': [100, 500, 1000]
}

In [14]:
# Set up gridsearch
lr_tv_gs = GridSearchCV(lr_tv_pipe, lr_tv_params, n_jobs=4, cv=5, verbose=1)

# Fit the gridsearch to the training data
lr_tv_gs.fit(X_train, y_train);

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.0min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  4.8min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 10.4min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 18.8min
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed: 29.7min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed: 44.6min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed: 61.9min
[Parallel(n_jobs=4)]: Done 2880 out of 2880 | elapsed: 74.0min finished


In [15]:
# What was our highest cross-val accuracy score across model hyperparameter combinations?
lr_tv_gs.best_score_

0.9947781732484282

In [16]:
# What model hyperparameters yielded the highest accuracy score?
lr_tv_gs.best_estimator_

Pipeline(memory=None,
         steps=[('tvec',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=500,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('lr',
                 LogisticRegression(C=1000000000.0, class_weight=None,
                                    dual=False, fit_intercept=True,
           

In [17]:
# What were the parameters?
lr_tv_gs.best_params_

{'lr__C': 1000000000.0,
 'lr__max_iter': 100,
 'lr__penalty': 'l2',
 'tvec__max_df': 1.0,
 'tvec__max_features': 500,
 'tvec__min_df': 1,
 'tvec__ngram_range': (1, 2),
 'tvec__stop_words': None}

In [18]:
# How does this model perform on the test set?
lr_tv_gs.score(X_test, y_test)

0.9948911648628362

## Support Vector Machine: CountVectorizer

In [21]:
# Set up pipeline
svc_cv_pipe = Pipeline([
    ('cvec', CountVectorizer()), 
    ('svc', SVC(gamma='scale')) 
])

# Set up pipeline parameters
svc_cv_params = {
    'cvec__stop_words': [None, 'english'],
    'cvec__ngram_range': [(1,1), (1,2), (1,3)], 
    'cvec__max_features': [100, 250, 500],
    'svc__C': [1, 2, 3, 5.5, 7, 8.5, 10, 20, 50], 
}

In [22]:
# Set up gridsearch
svc_cv_gs = GridSearchCV(svc_cv_pipe, svc_cv_params, n_jobs=4, cv=5, verbose=1)

# Fit the gridsearch to the training data
svc_cv_gs.fit(X_train, y_train);

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 16.8min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 62.9min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 161.1min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 332.8min
[Parallel(n_jobs=4)]: Done 810 out of 810 | elapsed: 339.4min finished


In [23]:
# What was our highest cross-val accuracy score across model hyperparameter combinations?
svc_cv_gs.best_score_

0.9965537557605546

In [24]:
# What model hyperparameters yielded the highest accuracy score?
svc_cv_gs.best_estimator_

Pipeline(memory=None,
         steps=[('cvec',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=500, min_df=1, ngram_range=(1, 2),
                                 preprocessor=None, stop_words='english',
                                 strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('svc',
                 SVC(C=3, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='scale',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     ver

In [25]:
# What were the parameters?
svc_cv_gs.best_params_

{'cvec__max_features': 500,
 'cvec__ngram_range': (1, 2),
 'cvec__stop_words': 'english',
 'svc__C': 3}

In [26]:
# How does this model perform on the test set?
svc_cv_gs.score(X_test, y_test)

0.9970702888549914

## Support Vector Machine: TfidfVectorizer

In [27]:
# Set up pipeline
svc_tv_pipe = Pipeline([
    ('tvec', TfidfVectorizer()), 
    ('svc', SVC(gamma='scale')) 
])

# Set up pipeline parameters
svc_tv_params = {
    'tvec__stop_words': [None, 'english'],
    'tvec__ngram_range': [(1,1), (1,2), (1,3)], 
    'tvec__max_features': [100, 250, 500],
    'svc__C': [1, 2, 3, 5.5, 7, 8.5, 10, 20, 50], 
}

In [28]:
# Set up gridsearch
svc_tv_gs = GridSearchCV(svc_tv_pipe, svc_tv_params, n_jobs=4, cv=5, verbose=1)

# Fit the gridsearch to the training data
svc_tv_gs.fit(X_train, y_train);

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  9.8min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 52.0min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 129.4min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 236.8min
[Parallel(n_jobs=4)]: Done 810 out of 810 | elapsed: 242.9min finished


In [29]:
# What was our highest cross-val accuracy score across model hyperparameter combinations?
svc_tv_gs.best_score_

0.9970460763661897

In [30]:
# What model hyperparameters yielded the highest accuracy score?
svc_tv_gs.best_estimator_

Pipeline(memory=None,
         steps=[('tvec',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=500,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svc',
                 SVC(C=7, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='scale',


In [31]:
# What were the parameters?
svc_tv_gs.best_params_

{'svc__C': 7,
 'tvec__max_features': 500,
 'tvec__ngram_range': (1, 1),
 'tvec__stop_words': None}

In [32]:
# How does this model perform on the test set?
svc_tv_gs.score(X_test, y_test)

0.9973366262318104

## Import Test Data

In [7]:
# Import test data to get project predictions
test_df = pd.read_csv('./data/test_data/combined_test_data.csv')
test_df.shape

(3096, 5)

In [8]:
# Look at the first 5 rows
test_df.head()

,Unnamed: 0,0,1,2,3
0,0,2017-08-31 21:34:27+00:00,DallasPD,Reminder: Texas State Law prohibiting texting ...,NaN
1,1,2017-08-31 21:03:18+00:00,DallasPD,Brotherhood for the Fallen Charity Event,NaN
2,2,2017-08-31 17:16:43+00:00,DallasPD,FREE #TRINITY EVENT - Artisan Fair Market - Se...,#TRINITY #LaborDayWeekend
3,3,2017-08-31 17:13:36+00:00,DallasPD,Our thoughts and prayers are with the Sacramen...,NaN
4,4,2017-08-31 17:10:47+00:00,DallasPD,How can Dallas residents help animals affected...,#HurricaneHarvey


In [9]:
# Drop 'Unnamed: 0' and hashtage ('3') columns
test_df.drop(columns=['Unnamed: 0', '3'], inplace=True)
test_df.head()

,0,1,2
0,2017-08-31 21:34:27+00:00,DallasPD,Reminder: Texas State Law prohibiting texting ...
1,2017-08-31 21:03:18+00:00,DallasPD,Brotherhood for the Fallen Charity Event
2,2017-08-31 17:16:43+00:00,DallasPD,FREE #TRINITY EVENT - Artisan Fair Market - Se...
3,2017-08-31 17:13:36+00:00,DallasPD,Our thoughts and prayers are with the Sacramen...
4,2017-08-31 17:10:47+00:00,DallasPD,How can Dallas residents help animals affected...


In [10]:
# Rename columns
test_df.rename(columns={'0': 'dates&time', '1': 'user', '2': 'tweet'}, inplace=True)
test_df.head()

,dates&time,user,tweet
0,2017-08-31 21:34:27+00:00,DallasPD,Reminder: Texas State Law prohibiting texting ...
1,2017-08-31 21:03:18+00:00,DallasPD,Brotherhood for the Fallen Charity Event
2,2017-08-31 17:16:43+00:00,DallasPD,FREE #TRINITY EVENT - Artisan Fair Market - Se...
3,2017-08-31 17:13:36+00:00,DallasPD,Our thoughts and prayers are with the Sacramen...
4,2017-08-31 17:10:47+00:00,DallasPD,How can Dallas residents help animals affected...


In [11]:
# Check for nulls
test_df.isnull().sum()

dates&time    0
user          0
tweet         9
dtype: int64

In [12]:
# Remove rows will null values for 'tweet' and verify changes
test_df = test_df[test_df['tweet'].notnull()]
test_df.isnull().sum()

dates&time    0
user          0
tweet         0
dtype: int64

In [13]:
# Export clean test data
test_df.to_csv('./data/test_data/final_test_data.csv', index=False)

## Pickle / Export Models

In [14]:
import pickle

### SVC / TfidfVectorizer

In [37]:
# Instantiate and fit svc pipeline with best hyperparameters
best_pipe = Pipeline([
    ('tvec', TfidfVectorizer(max_features=500)),
    ('svc', SVC(gamma='scale', C=7))
])
best_pipe.fit(X_train, y_train);

In [39]:
# Get the score on train and test data
print(f'Train score: {best_pipe.score(X_train, y_train)}')
print(f'Train cross val score: {cross_val_score(best_pipe, X_train, y_train, cv=5).mean()}')
print(f'Test score: {best_pipe.score(X_test, y_test)}')

Train score: 0.9999273625335948
Train cross val score: 0.9970460777980892
Test score: 0.9973366262318104


In [40]:
# Get predictions on test set of train data
y_pred = best_pipe.predict(X_test)
y_pred[:5]

array([0, 0, 1, 0, 0])

In [86]:
# Define X and y for test data
test_X = test_df['tweet']

# Get predicitons on our true test data
test_y_pred = best_pipe.predict(test_X)
test_y_pred[:5]

array([0, 0, 0, 0, 0])

In [87]:
# Set filename and export
filename = './assets/svc_tv.sav'
pickle.dump(best_pipe, open(filename, 'wb'))

In [88]:
# Get predictions on our true test data using the pickled model
model = pickle.load(open('svc_tv.sav','rb'))
print(model.predict(test_X)[:5])

[0 0 0 0 0]


### Logistic Regression / CountVectorizer

In [15]:
# Instantiate and fit svc pipeline with best hyperparameters
best_pipe_lr = Pipeline([
    ('cvec', CountVectorizer(max_features=500, ngram_range=(1,2))),
    ('lr', LogisticRegression(solver='liblinear', penalty='l1'))
])
best_pipe_lr.fit(X_train, y_train);

In [16]:
# Get the score on train and test data
print(f'Train score: {best_pipe_lr.score(X_train, y_train)}')
print(f'Train cross val score: {cross_val_score(best_pipe_lr, X_train, y_train, cv=5).mean()}')
print(f'Test score: {best_pipe_lr.score(X_test, y_test)}')

Train score: 0.9979822925998564
Train cross val score: 0.9971429268413905
Test score: 0.9972882012542069


In [17]:
# Get predictions on test set of train data
y_pred_lr = best_pipe_lr.predict(X_test)
y_pred_lr[:5]

array([0, 0, 1, 0, 0])

In [18]:
# Define X and y for test data
test_X = test_df['tweet']

# Get predicitons on our true test data
test_y_pred_lr = best_pipe_lr.predict(test_X)
test_y_pred_lr[:5]

array([0, 0, 0, 0, 0])

In [68]:
# Set filename and export
filename = './assets/lr_cv.sav'
pickle.dump(best_pipe_lr, open(filename, 'wb'))

In [90]:
# Get predictions on our true test data using the pickled model
model_lr = pickle.load(open('./assets/lr_cv.sav','rb'))
print(model_lr.predict(test_X)[:5])

[0 0 0 0 0]


## Compare Test Data Predictions

In [91]:
pred_comparison = list(zip(model.predict(test_X), model_lr.predict(test_X)))
pred_comparison[:5]

[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]

In [92]:
# How many tweets are labelled differently between the two models?
count = 0
for tup in pred_comparison:
    svc, lr = tup
    if svc != lr:
        count +=1
count

28

In [93]:
# How many tweets between the two models were labelled useful?
count = 0
for tup in pred_comparison:
    svc, lr = tup
    if svc == 1 or lr == 1:
        count +=1
count

126

In [94]:
# How many tweets were labelled useful in both models?
count = 0
for tup in pred_comparison:
    svc, lr = tup
    if svc == 1 and lr == 1:
        count +=1
count

98

## Model Evaluation

In [19]:
# Get y predictions on test set of train data
model_lr = pickle.load(open('./assets/lr_cv.sav','rb'))
y_pred = model_lr.predict(X_test)
print(model_lr.predict(X_test)[:5])

[0 0 1 0 0]


In [20]:
# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, 
             columns=['predicted not useful', 'predicted useful'], 
             index=['actual not useful', 'actual useful'])
cm_df

,predicted not useful,predicted useful
actual not useful,39115,73
actual useful,39,2074


In [23]:
# Get tn, fp, fn, tp
tn, fp, fn, tp = cm.ravel()

# Calculate sensitivity
print(f'Sensitivity: {tp / (tp + fn)}')

# Calculate specificity
print(f'Specificity: {tn / (tn + fp)}')

# Calculate precision
print(f'Precision: {tp / (tp + fp)}')

# Calculate false negative rate
print(f'False negative rate: {tn / (tn + fn)}')

# Calculate accuracy
print(f'Accuracy: {(tp + tn) / (tp + tn + fp + fn)}')


Sensitivity: 0.9815428300993848
Specificity: 0.9981371848525059
Precision: 0.9659990684676293
False negative rate: 0.999003933186903
Accuracy: 0.9972882012542069


## Explore Results

In [28]:
# Create dataframe of test set of train data
explore_df = pd.DataFrame(X_test)
explore_df = pd.concat([explore_df, y_test], axis=1)
explore_df.head()

,tweet,class
63790,Teen CPA class night pictures Teens learned a...,0
136287,Preparation for the SH at SH groundbreaking ...,0
6656,The access roads on I closed both ways at the ...,1
93714,The sunny weather weve enjoyed this week is no...,0
84860,Neither the trooper or the suspects were injur...,0


In [29]:
# Add predictions column
explore_df['pred'] = y_pred
explore_df.head()

,tweet,class,pred
63790,Teen CPA class night pictures Teens learned a...,0,0
136287,Preparation for the SH at SH groundbreaking ...,0,0
6656,The access roads on I closed both ways at the ...,1,1
93714,The sunny weather weve enjoyed this week is no...,0,0
84860,Neither the trooper or the suspects were injur...,0,0


In [31]:
# Create a column indicating misclassified tweet
correct = []
for x, y in zip(y_test, y_pred):
    if x == y:
        correct.append(1)
    else:
        correct.append(0)
# Verify that list contains both 112 0s
count = 0
for num in correct:
    if num == 0:
        count += 1
count

112

In [32]:
# Return a dataframe of misclassified tweets
explore_df['correct'] = correct
misclassified = explore_df[explore_df['correct'] == 0]

# Set view options and look at misclassified tweets
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
misclassified

,tweet,class,pred,correct
5966,SH in Winkler County closed due to crash Traffic diverted at Highway and at truck route in Kermit,1,0,0
6141,Goat traffic jam In the Devils River bridge on US Highway near DelRio closed to allow goats to be driven across TxDOT,1,0,0
6753,FM closed in Bastrop Co due to wheeler crash at High Crossing ATXTraffic,1,0,0
47873,MERGE LanesDo not slow to a near stop or stop and abruptly cut across into traffic Instead use the merge lane to smoothly transition into the lane of travel Drivers in the traffic lane should maintain their lane and not cut off the merge lane,0,1,0
143370,Thank you Is is now closed Be safe,0,1,0
140685,FM closed for an undetermined amt of timeTxDOT working on emergency contract Dont movego around barricades,0,1,0
143668,REMINDER Anthony overpass is closed tonight for bridge work Plan to exit Vinton then take Doniphan,0,1,0
162,Traffic Advisory APD Traffic Investigation Unit has west bound IH closed from Georgia to Western,1,0,0
146368,NB IW at CR in Johnson County is closed due to a wreck Through traffic should use E,0,1,0
111523,Milo Interchange direct connector currently closed until Feb for concrete repair Please use alternate routes,0,1,0
